This script aims to convert json file into csv file


In [12]:
import json
import os
import pandas as pd

### 1. Load data

In [5]:
# Define the path to the data folder relative to the notebook
current_dir = os.getcwd()  # Get the current working directory
input_path = os.path.join(current_dir, '../data/mobidata/raw/')  # Adjust path to the data folder

input_file = 'baustellen_vorschau.json'
input_file_path = os.path.join(input_path, input_file)

In [8]:
with open(input_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [13]:
# High-level info
print(type(data))
print("Top-level keys:", list(data.keys()))
print("totalFeatures:", data.get("totalFeatures"))
print("numberReturned:", data.get("numberReturned"))
print("timeStamp:", data.get("timeStamp"))

<class 'dict'>
Top-level keys: ['type', 'features', 'totalFeatures', 'numberMatched', 'numberReturned', 'timeStamp', 'crs']
totalFeatures: 270
numberReturned: 270
timeStamp: 2025-10-06T14:15:33.080Z


In [ ]:
# Features summary and keys
features = data.get("features", [])
print(f"Number of features: {len(features)}")
if features:
    f0 = features[0]
    print("Feature[0] keys:", list(f0.keys()))
    print("geometry keys:", list(f0.get("geometry", {}).keys()))
    print("properties keys (sample):", list(f0.get("properties", {}).keys()))

In [ ]:
# Pretty-print the first feature (readable JSON)
if features:
    print(json.dumps(features[0], ensure_ascii=False, indent=2))

In [ ]:
# Optional: tabular view of properties for the first N features
N = 10
props = [feat.get("properties", {}) for feat in features[:N]]
df = pd.json_normalize(props)
df.head()

In [10]:


# quick inspect
if isinstance(data, list):
    print(f'Loaded list with {len(data)} items. Sample:', data[:2])
elif isinstance(data, dict):
    print(f'Loaded dict with keys: {list(data.keys())}')
else:
    print('Loaded JSON of type', type(data))

# # optionally save a pretty-printed JSON to output_path
# Path(output_path).write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding='utf-8')
# print(f'Wrote pretty JSON to {output_path}')


Loaded dict with keys: ['type', 'features', 'totalFeatures', 'numberMatched', 'numberReturned', 'timeStamp', 'crs']
